# Image processing

## Load an image to numpy

In [4]:
import PIL
from PIL import Image
import numpy as np
print('Pillow Version:', PIL.__version__)
import pandas as pd

Pillow Version: 8.1.0


In [6]:
veg_img = Image.open('vegetation2020.png').convert('P')
palette = veg_img.getpalette()
palette_arr = np.asarray(palette).reshape(256, 3)
veg_img_arr = np.asarray(veg_img)
print(f"shape: {veg_img_arr.shape} dtype: {veg_img_arr.dtype}")
print(f"palette: {palette_arr.shape} dtype: {palette_arr.dtype}")

shape: (1080, 1920) dtype: uint8
palette: (256, 3) dtype: int64


## Understand the palette

We can check the how many colors from the palette are been used in the entire image. For vegetation2020, it is a palette of 30 colors.

In [8]:
np.unique(veg_img_arr)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=uint8)

In [9]:
palette_arr[0:30]

array([[254, 254, 254],
       [  2,   2,   2],
       [232, 232, 232],
       [215, 215, 215],
       [ 23,  23,  23],
       [ 55,  71,  97],
       [199, 199, 200],
       [183, 183, 183],
       [ 39,  39,  39],
       [167, 167, 168],
       [ 55,  55,  55],
       [135, 135, 136],
       [151, 151, 151],
       [ 71,  71,  71],
       [119, 119, 119],
       [ 87,  87,  87],
       [103, 103, 103],
       [ 72,  87, 110],
       [142, 151, 165],
       [107, 119, 138],
       [ 87, 101, 122],
       [180, 186, 195],
       [122, 132, 150],
       [188, 194, 202],
       [155, 163, 176],
       [217, 220, 225],
       [ 97, 109, 130],
       [222, 225, 229],
       [ 95, 107, 128],
       [ 64,  79, 104]])

In [11]:
color_id, color_frequency = np.unique(veg_img_arr, return_counts = True)
color_frequency

array([1882788,   52211,   17195,   12337,   10959,   10353,   10175,
          8440,    8384,    7944,    7288,    7085,    6974,    6744,
          6633,    6514,    6427,     846,     789,     674,     526,
           507,     474,     363,     352,     305,     118,      94,
            62,      39])

In [12]:
## Change the palette

In [13]:
img_exp_1 = Image.fromarray(veg_img_arr, 'P')
img_exp_1.putpalette(palette)
img_exp_1.save('img-exp-1.png')